# tensorflow_advanced_segmentation_models Experimentation

**NOPE, this doesn't seem to work.**

This notebook's code was based on https://github.com/JanMarcelKezmann/TensorFlow-Advanced-Segmentation-Models/blob/master/examples/TASM_Example_1.ipynb and ChatGPT explanations.

## Imports

In [1]:
# Standard libraries
import os
import json
import random
import shutil
from time import time

# Third-party libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# Deep learning and augmentation libraries
import tensorflow as tf
import tensorflow.keras.backend as K
import albumentations as A
import tensorflow_advanced_segmentation_models as tasm

## Split dataset to train-val-test

In [2]:
# # Input directories
# damaged_img_dir = '../../data/damaged-and-mask-dataset/generated-damaged-images'
# damage_mask_dir = '../../data/damaged-and-mask-dataset/generated-damage-masks'

# # Output directories
# train_img_dir = '../../data/damaged-and-mask-dataset/img-with-val/train'
# val_img_dir   = '../../data/damaged-and-mask-dataset/img-with-val/val'
# test_img_dir  = '../../data/damaged-and-mask-dataset/img-with-val/test'

# train_mask_dir = '../../data/damaged-and-mask-dataset/mask-with-val/train'
# val_mask_dir   = '../../data/damaged-and-mask-dataset/mask-with-val/val'
# test_mask_dir  = '../../data/damaged-and-mask-dataset/mask-with-val/test'

# # Create directories if they don't exist
# for d in [train_img_dir, val_img_dir, test_img_dir, train_mask_dir, val_mask_dir, test_mask_dir]:
#     os.makedirs(d, exist_ok=True)

# # Get and shuffle list of image filenames
# all_images = [f for f in os.listdir(damaged_img_dir) if f.lower().endswith('.png')]
# random.seed(42)
# random.shuffle(all_images)

# # Compute split indices
# n_total = len(all_images)
# n_train = int(0.70 * n_total)
# n_val = int(0.15 * n_total)
# n_test = n_total - n_train - n_val

# # Split images
# train_images = all_images[:n_train]
# val_images = all_images[n_train:n_train + n_val]
# test_images = all_images[n_train + n_val:]

# # Copy function
# def copy_files(img_list, target_img_dir, target_mask_dir):
#     for fname in img_list:
#         src_img_path = os.path.join(damaged_img_dir, fname)
#         dst_img_path = os.path.join(target_img_dir, fname)
#         shutil.copy(src_img_path, dst_img_path)

#         base_name = fname.replace('.png', '')
#         mask_name = f'{base_name}-mask.png'
#         src_mask_path = os.path.join(damage_mask_dir, mask_name)
#         dst_mask_path = os.path.join(target_mask_dir, mask_name)
#         if os.path.exists(src_mask_path):
#             shutil.copy(src_mask_path, dst_mask_path)
#         else:
#             print(f'Warning: mask for {fname} not found')

# # Perform copying
# copy_files(train_images, train_img_dir, train_mask_dir)
# copy_files(val_images, val_img_dir, val_mask_dir)
# copy_files(test_images, test_img_dir, test_mask_dir)

# print(f"Split complete: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test")


In [3]:
# # Directories for training set
# img_train_dir = '../../data/damaged-and-mask-dataset/img/train'
# mask_train_dir = '../../data/damaged-and-mask-dataset/mask/train'

# # List all image and mask files
# damaged_train = [f for f in os.listdir(img_train_dir) if os.path.isfile(os.path.join(img_train_dir, f))]
# mask_train = [f for f in os.listdir(mask_train_dir) if os.path.isfile(os.path.join(mask_train_dir, f))]

# # Print samples
# print(f'   Sample Mask Files (Train) \n{mask_train[:5]}')
# print(f'   Sample Damaged Image Files (Train) \n{damaged_train[:5]}')

## Directories for Damaged and Mask Images

In [4]:
# Data Handling
DATA_DIR = "../../data/"

x_train_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/img-with-val/train")
y_train_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/mask-with-val/train")

x_valid_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/img-with-val/val")
y_valid_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/mask-with-val/val")

x_test_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/img-with-val/test/")
y_test_dir = os.path.join(DATA_DIR, "damaged-and-mask-dataset/mask-with-val/test/")

In [5]:
# # Define the directories of the paintings and masks
# IMAGE_DIR = "../../data/damaged-and-mask-dataset/generated-damaged-images"
# MASK_DIR = "../../data/damaged-and-mask-dataset/generated-damage-masks"

# # Clean up the filenames of the paintings and masks
# image_filenames = sorted([f for f in os.listdir(IMAGE_DIR) if f.endswith(".png")])
# mask_filenames = [f.replace(".png", "-mask.png") for f in image_filenames]

# # Define the filepaths
# image_paths = [os.path.join(IMAGE_DIR, f) for f in image_filenames]
# mask_paths = [os.path.join(MASK_DIR, f) for f in mask_filenames]

# # Split the dataset into train-val-test
# x_train_dir, X_temp_dir, y_train_dir, y_temp_dir = train_test_split(image_paths, mask_paths, 
#                                                     test_size=0.3, random_state=42)

# x_valid_dir, x_test_dir, y_valid_dir, y_test_dir = train_test_split(X_temp_dir, y_temp_dir, 
#                                                 test_size = 0.5, random_state=42)

## Helper Functions

In [6]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()


# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x


def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)
     

## Data Augmentation Functions

In [7]:
# define heavy augmentations
def get_training_augmentation(height, width):
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.Affine(scale_limit=0.6, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        A.PadIfNeeded(min_height=height, min_width=width, border_mode=0),
        A.RandomCrop(height=height, width=width),

        A.GaussNoise(p=0.2),  # Already correct
        A.Perspective(scale=(0.05, 0.1), p=0.5),  # Replacement for IAAPerspective

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0, p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.Sharpen(p=1),  # Replacement for IAASharpen
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=0.2, p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)


def get_validation_augmentation(height, width):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(height, width),
        A.Resize(height, width)
    ]
    return A.Compose(test_transform)


def data_get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

## Define some global variables

In [8]:
TOTAL_CLASSES = ["painting", "damage_mask"]
N_TOTAL_CLASSES = 2

with open("class_pixel_counts.json", "r") as f:
    CLASSES_PIXEL_COUNT_DICT = json.load(f)

MODEL_CLASSES = TOTAL_CLASSES
BATCH_SIZE = 4
N_CLASSES = 1 # Binary segmentation: pixel is either damage or painting
HEIGHT = 320
WIDTH = 320
BACKBONE_NAME = "resnet50"
WEIGHTS = "imagenet"
WWO_AUG = True # Train data with and without augmentation

## Functions to calculate appropriate class weights

In [9]:
################################################################################
# Class Weights
################################################################################
def get_dataset_counts(d):
    pixel_count = np.array([i for i in d.values()])

    sum_pixel_count = 0
    for i in pixel_count:
        sum_pixel_count += i

    return pixel_count, sum_pixel_count


def get_dataset_statistics(pixel_count, sum_pixel_count):
    
    pixel_frequency = np.round(pixel_count / sum_pixel_count, 4)

    mean_pixel_frequency = np.round(np.mean(pixel_frequency), 4)

    return pixel_frequency, mean_pixel_frequency


def get_balancing_class_weights(classes, d):
    pixel_count, sum_pixel_count = get_dataset_counts(d)

    background_pixel_count = 0
    mod_pixel_count = []
    for c in TOTAL_CLASSES:
        if c not in classes:
            background_pixel_count += d[c]
        else:
            mod_pixel_count.append(d[c])

    if background_pixel_count > 0:
        mod_pixel_count.append(background_pixel_count)

    pixel_frequency, mean_pixel_frequency = get_dataset_statistics(mod_pixel_count, sum_pixel_count)

    # Avoid division by zero
    pixel_frequency = np.where(pixel_frequency == 0, 1e-8, pixel_frequency)

    class_weights = np.round(mean_pixel_frequency / pixel_frequency, 2)
    return class_weights


class_weights = get_balancing_class_weights(MODEL_CLASSES, CLASSES_PIXEL_COUNT_DICT)
print(class_weights)

[0.55 5.82]


## Data Generation Functions

In [10]:
################################################################################
# Data Generator
################################################################################
def create_image_label_path_generator(images_dir, masks_dir, shuffle=False, seed=None):
    ids = sorted(os.listdir(images_dir))
    mask_ids = sorted(os.listdir(masks_dir))

    if shuffle == True:

        if seed is not None:
            tf.random.set_seed(seed)

        indices = tf.range(start=0, limit=tf.shape(ids)[0], dtype=tf.int32)
        shuffled_indices = tf.random.shuffle(indices)

        ids = tf.gather(ids, shuffled_indices).numpy().astype(str)
        mask_ids = tf.gather(mask_ids, shuffled_indices).numpy().astype(str)

    images_fps = [os.path.join(images_dir, image_id) for image_id in ids]
    masks_fps = [os.path.join(masks_dir, image_id) for image_id in mask_ids]

    while True:
        for i in range(len(images_fps)):
            yield [images_fps[i], masks_fps[i]]


def process_image_label(images_paths, masks_paths, classes, augmentation=None, preprocessing=None):
    class_values = [TOTAL_CLASSES.index(cls.lower()) for cls in classes]
    
    # read data
    image = cv2.imread(images_paths)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(masks_paths, 0)

    # extract certain classes from mask (e.g. cars)
    masks = [(mask == v) for v in class_values]
    mask = np.stack(masks, axis=-1).astype('float')
    
    # add background if mask is not binary
    if mask.shape[-1] != 1:
        background = 1 - mask.sum(axis=-1, keepdims=True)
        mask = np.concatenate((mask, background), axis=-1)
    
    # apply augmentations
    if augmentation:
        sample = augmentation(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']
    
    # apply preprocessing
    if preprocessing:
        sample = preprocessing(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']

    # mask = np.squeeze(np.argmax(mask, axis=-1))
    # mask = np.argmax(mask, axis=-1)
    # mask = mask[..., np.newaxis]
        
    return image, mask

def DataGenerator(train_dir, label_dir, batch_size, height, width, classes, augmentation, wwo_aug=False, shuffle=False, seed=None):
    image_label_path_generator = create_image_label_path_generator(
        train_dir, label_dir, shuffle=shuffle, seed=seed
    )
    if wwo_aug:
        while True:
            images = np.zeros(shape=[batch_size, height, width, 3])
            labels = np.zeros(shape=[batch_size, height, width, len(classes) + 1], dtype=np.float32)
            for i in range(0, batch_size, 2):
                image_path, label_path = next(image_label_path_generator)
                image_aug, label_aug = process_image_label(image_path, label_path, classes=classes, augmentation=augmentation)
                image_wo_aug, label_wo_aug = process_image_label(image_path, label_path, classes=classes, augmentation=get_validation_augmentation(height=HEIGHT, width=WIDTH))
                images[i], labels[i] = image_aug, label_aug
                images[i + 1], labels[i + 1] = image_wo_aug, label_wo_aug

            yield tf.convert_to_tensor(images), tf.convert_to_tensor(labels, tf.float32)
    else:
        while True:
            images = np.zeros(shape=[batch_size, height, width, 3])
            labels = np.zeros(shape=[batch_size, height, width, len(classes) + 1], dtype=np.float32)
            for i in range(batch_size):
                image_path, label_path = next(image_label_path_generator)
                image, label = process_image_label(image_path, label_path, classes=classes, augmentation=augmentation)
                images[i], labels[i] = image, label

            yield tf.convert_to_tensor(images), tf.convert_to_tensor(labels, tf.float32)

In [11]:
# ################################################################################
# # Data Generator
# ################################################################################
# def create_image_label_path_generator(image_paths, mask_paths, shuffle=False, seed=None):
#     if shuffle:
#         combined = list(zip(image_paths, mask_paths))
#         rng = np.random.default_rng(seed)
#         rng.shuffle(combined)
#         image_paths, mask_paths = zip(*combined)

#     for img_path, mask_path in zip(image_paths, mask_paths):
#         yield img_path, mask_path


# def process_image_label(images_paths, masks_paths, classes, augmentation=None, preprocessing=None):
#     class_values = [TOTAL_CLASSES.index(cls.lower()) for cls in classes]
    
#     # read data
#     image = cv2.imread(images_paths)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     mask = cv2.imread(masks_paths, 0)

#     # extract certain classes from mask (e.g. cars)
#     masks = [(mask == v) for v in class_values]
#     mask = np.stack(masks, axis=-1).astype('float')
    
#     # add background if mask is not binary
#     if mask.shape[-1] != 1:
#         background = 1 - mask.sum(axis=-1, keepdims=True)
#         mask = np.concatenate((mask, background), axis=-1)
    
#     # apply augmentations
#     if augmentation:
#         sample = augmentation(image=image, mask=mask)
#         image, mask = sample['image'], sample['mask']
    
#     # apply preprocessing
#     if preprocessing:
#         sample = preprocessing(image=image, mask=mask)
#         image, mask = sample['image'], sample['mask']

#     # mask = np.squeeze(np.argmax(mask, axis=-1))
#     # mask = np.argmax(mask, axis=-1)
#     # mask = mask[..., np.newaxis]
        
#     return image, mask


# def DataGenerator(train_dir, label_dir, batch_size, height, width, classes, augmentation, wwo_aug=False, shuffle=False, seed=None):
#     image_label_path_generator = create_image_label_path_generator(
#         train_dir, label_dir, shuffle=shuffle, seed=seed
#     )
#     if wwo_aug:
#         while True:
#             images = np.zeros(shape=[batch_size, height, width, 3])
#             labels = np.zeros(shape=[batch_size, height, width, len(classes) + 1], dtype=np.float32)
#             for i in range(0, batch_size, 2):
#                 image_path, label_path = next(image_label_path_generator)
#                 image_aug, label_aug = process_image_label(image_path, label_path, classes=classes, augmentation=augmentation)
#                 image_wo_aug, label_wo_aug = process_image_label(image_path, label_path, classes=classes, augmentation=get_validation_augmentation(height=HEIGHT, width=WIDTH))
#                 images[i], labels[i] = image_aug, label_aug
#                 images[i + 1], labels[i + 1] = image_wo_aug, label_wo_aug

#             yield tf.convert_to_tensor(images), tf.convert_to_tensor(labels, tf.float32)
#     else:
#         while True:
#             images = np.zeros(shape=[batch_size, height, width, 3])
#             labels = np.zeros(shape=[batch_size, height, width, len(classes) + 1], dtype=np.float32)
#             for i in range(batch_size):
#                 image_path, label_path = next(image_label_path_generator)
#                 image, label = process_image_label(image_path, label_path, classes=classes, augmentation=augmentation)
#                 images[i], labels[i] = image, label

#             yield tf.convert_to_tensor(images), tf.convert_to_tensor(labels, tf.float32)

## Create the Model

**NOTE:** Change to U-Net, if necessary.

In [12]:
base_model, layers, layer_names = tasm.create_base_model(name=BACKBONE_NAME, weights=WEIGHTS, height=HEIGHT, width=WIDTH, include_top=False, pooling=None)

BACKBONE_TRAINABLE = False
model = tasm.UNet(n_classes=N_CLASSES, base_model=base_model, output_layers=layers, backbone_trainable=BACKBONE_TRAINABLE)

### Define the optimizer, losses, metrics, and callbacks

In [13]:
# Define optimizer (Adam adapts learning rates per parameter)
opt = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
    epsilon=1e-7
)

# Use IoU as the primary performance metric
metrics = [tasm.metrics.IOUScore(threshold=0.5)]

# Combine Binary Focal Loss (for class imbalance) with Dice Loss (for region overlap)
binary_focal_dice_loss = (
    tasm.losses.BinaryFocalLoss(alpha=0.25, gamma=2.0) +
    tasm.losses.DiceLoss()
)

# Compile the model with optimizer, loss function, and evaluation metric
model.compile(
    optimizer=opt,
    loss=binary_focal_dice_loss,
    metrics=metrics,
)

# Run model in eager mode (easier debugging, slightly slower)
model.run_eagerly = True

# Define training callbacks
callbacks = [
    # Save full model (architecture + weights) only when IoU improves
    tf.keras.callbacks.ModelCheckpoint(
        filepath="models/DeepLabv3plus_full_model.keras",
        monitor="iou_score",
        mode="max",
        verbose=1,
        save_best_only=True,
        save_weights_only=False
    ),

    # Stop training if no IoU improvement after 16 epochs, and restore best model
    tf.keras.callbacks.EarlyStopping(
        monitor="iou_score",
        patience=16,
        mode="max",
        verbose=1,
        restore_best_weights=True
    ),

    # Reduce learning rate by a factor of 0.2 if IoU doesn't improve after 6 epochs
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="iou_score",
        factor=0.2,
        patience=6,
        verbose=1,
        mode="max"
    ),

    # Log training progress for visualization in TensorBoard
    tf.keras.callbacks.TensorBoard(
        log_dir="logs"
    )
]

## Get the Data

In [14]:
train_shuffle = True
val_shuffle = True
seed = 29598

TrainSet = DataGenerator(
    x_train_dir,
    y_train_dir,
    BATCH_SIZE,
    HEIGHT,
    WIDTH,
    classes=MODEL_CLASSES,
    augmentation=get_training_augmentation(height=HEIGHT, width=WIDTH),
    shuffle=train_shuffle,
    seed=seed
    )

TrainSetwoAug = DataGenerator(
    x_train_dir,
    y_train_dir,
    BATCH_SIZE,
    HEIGHT,
    WIDTH,
    classes=MODEL_CLASSES,
    augmentation=get_validation_augmentation(height=HEIGHT, width=WIDTH),
    shuffle=train_shuffle,
    seed=seed
)

TrainSetwwoAug = DataGenerator(
    x_train_dir,
    y_train_dir,
    BATCH_SIZE,
    HEIGHT,
    WIDTH,
    classes=MODEL_CLASSES,
    augmentation=get_training_augmentation(height=HEIGHT, width=WIDTH),
    wwo_aug=True,
    shuffle=train_shuffle,
    seed=seed
)

ValidationSet = DataGenerator(
    x_valid_dir,
    y_valid_dir,
    1,
    HEIGHT,
    WIDTH,
    classes=MODEL_CLASSES,
    augmentation=get_validation_augmentation(height=HEIGHT, width=WIDTH),
    shuffle=val_shuffle,
    seed=seed
)

TestSet = DataGenerator(
    x_test_dir,
    y_test_dir,
    1,
    HEIGHT,
    WIDTH,
    classes=MODEL_CLASSES,
    augmentation=get_validation_augmentation(height=HEIGHT, width=WIDTH),
)

for i in TrainSet:
    sample_image, sample_mask = i[0][0], i[1][0]
    print(len(i))
    print(i[0].shape)
    print(i[1].shape)
    break

print(len(os.listdir(x_train_dir)))
print(len(os.listdir(x_valid_dir)))
print(len(os.listdir(x_test_dir)))

/tmp/ipykernel_136857/2707607068.py:7: UserWarning: Argument(s) 'scale_limit, rotate_limit, shift_limit' are not valid for transform Affine
  A.Affine(scale_limit=0.6, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),


2
(4, 320, 320, 3)
(4, 320, 320, 3)
24205
5186
5188


Quick check if the model works properly.

In [15]:
def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
        plt.axis('off')
    plt.show()


def create_mask(pred_mask):
    # Convert sigmoid output to binary mask with threshold 0.5
    pred_mask = tf.where(pred_mask > 0.5, 1.0, 0.0)
    return pred_mask[0]


def show_predictions(model, sample_image, sample_mask, loss_fn):
    pred_output = model(sample_image[tf.newaxis, ...], training=False)
    pred_mask = create_mask(pred_output)

    bce = tf.keras.losses.BinaryCrossentropy()
    print("BinaryCrossentropy:", bce(sample_mask, pred_output[0]).numpy())
    print("IoU Score:", tasm.losses.iou_score(sample_mask, pred_output[0]).numpy())
    print("Binary Focal + Dice Loss:", loss_fn(sample_mask, pred_output[0]).numpy())

    display([sample_image, sample_mask, pred_mask])


In [16]:
tf.debugging.set_log_device_placement(True)

In [ ]:
for batch in ValidationSet:
    sample_image, sample_mask = batch[0][0], batch[1][0]
    break

show_predictions(model, sample_image, sample_mask, binary_focal_dice_loss)

In [ ]:
sample_mask

In [ ]:
tf.__version__